In [2]:
import requests
from bs4 import BeautifulSoup

url = "https://www.tradingview.com/markets/stocks-usa/market-movers-high-dividend/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

table = soup.find('table', class_='table-Ngq2xrcG')
rows = table.find_all('tr', attrs={'data-rowkey': True})

data_rowkeys = [row['data-rowkey'] for row in rows]
print(data_rowkeys)

['NASDAQ:RILY', 'NASDAQ:IEP', 'NASDAQ:GIPR', 'NYSE:MNR', 'NYSE:TPVG', 'NASDAQ:OFS', 'NASDAQ:TCPC', 'NASDAQ:DALN', 'NASDAQ:RWAY', 'NASDAQ:ICON', 'NASDAQ:PTMN', 'NYSE:VOC', 'NASDAQ:XP', 'NASDAQ:TRIN', 'NASDAQ:OXSQ', 'NYSE:KEN', 'NASDAQ:GECC', 'NASDAQ:PSEC', 'NASDAQ:NHTC', 'NYSE:CATO', 'NYSE:NEP', 'NASDAQ:OCSL', 'NYSE:GSBD', 'NASDAQ:WHF', 'NASDAQ:SQFT', 'NYSE:FSK', 'NYSE:SAR', 'NYSE:CMTG', 'NASDAQ:FAT', 'NYSE:DSX', 'NASDAQ:MRCC', 'NYSE:CION', 'NASDAQ:MNDO', 'NYSE:TXO', 'NASDAQ:USEA', 'AMEX:COHN', 'NYSE:SCM', 'NASDAQ:WBA', 'NYSE:KRP', 'NASDAQ:DMLP', 'NYSE:NAT', 'NASDAQ:ARLP', 'NASDAQ:SLRC', 'NASDAQ:NMFC', 'NYSE:GHI', 'NYSE:AMBP', 'NASDAQ:HTCR', 'NYSE:PSBD', 'NYSE:BBDC', 'NYSE:PFLT', 'NYSE:SGHC', 'NYSE:BXSL', 'NASDAQ:GBDC', 'NYSE:UAN', 'NYSE:KSS', 'NYSE:BCSF', 'NYSE:OBDC', 'NASDAQ:TTEC', 'NYSE:CAPL', 'NYSE:DKL', 'NYSE:BSM', 'NASDAQ:BGFV', 'NYSE:EVC', 'NYSE:OBDE', 'NASDAQ:XRX', 'NASDAQ:SBLK', 'NASDAQ:MARPS', 'NASDAQ:CGBD', 'NYSE:SFL', 'NYSE:BWMX', 'NYSE:USAC', 'NASDAQ:ARCC', 'NASDAQ:CCAP', '

In [3]:
symbols = [item.split(':')[1] for item in data_rowkeys]
print(symbols)


['RILY', 'IEP', 'GIPR', 'MNR', 'TPVG', 'OFS', 'TCPC', 'DALN', 'RWAY', 'ICON', 'PTMN', 'VOC', 'XP', 'TRIN', 'OXSQ', 'KEN', 'GECC', 'PSEC', 'NHTC', 'CATO', 'NEP', 'OCSL', 'GSBD', 'WHF', 'SQFT', 'FSK', 'SAR', 'CMTG', 'FAT', 'DSX', 'MRCC', 'CION', 'MNDO', 'TXO', 'USEA', 'COHN', 'SCM', 'WBA', 'KRP', 'DMLP', 'NAT', 'ARLP', 'SLRC', 'NMFC', 'GHI', 'AMBP', 'HTCR', 'PSBD', 'BBDC', 'PFLT', 'SGHC', 'BXSL', 'GBDC', 'UAN', 'KSS', 'BCSF', 'OBDC', 'TTEC', 'CAPL', 'DKL', 'BSM', 'BGFV', 'EVC', 'OBDE', 'XRX', 'SBLK', 'MARPS', 'CGBD', 'SFL', 'BWMX', 'USAC', 'ARCC', 'CCAP', 'BGS', 'CSWC', 'TSLX', 'WES', 'TFSL', 'BRY', 'FDUS', 'OMF', 'CBRL', 'HIHO', 'EPM', 'SPOK', 'WLKP', 'LIEN', 'PAX', 'AB', 'SHIP', 'GLAD', 'DHT', 'MO', 'AY', 'CVI', 'DSWL', 'VTS', 'HTGC', 'WU', 'ET']


As you can see below, the dividend capture stategy does not work. You have to change the buy and sell periods to essentially turn it into a buy-and-hold strategy for it to turn a small profit.

In [20]:
import pandas as pd
import yfinance as yf
from datetime import timedelta
import pandas_market_calendars as mcal
import warnings
from pandas.tseries.offsets import BDay

# Suppress warnings from yfinance about missing data
warnings.filterwarnings("ignore")

# Define your start and end dates
start_date = pd.Timestamp('2023-01-01')
end_date = pd.Timestamp('2024-10-01')

# List of stock symbols (use known dividend-paying stocks)
symbols = ['JEPQ', 'JEPI', 'SPYI', 'QQQI', 'DRGO']
capital = 100000  # Starting capital
before_days = 100
after_days = 100

# Get NYSE trading calendar
nyse = mcal.get_calendar('NYSE')
trading_days = nyse.valid_days(start_date=start_date, end_date=end_date)
trading_days = trading_days.tz_localize(None)  # Remove timezone info

# Convert trading_days to a set for faster lookup
trading_days_set = set(trading_days)

# Function to get the previous trading day
def get_previous_trading_day(date):
    min_date = trading_days[0]
    while date not in trading_days_set:
        date -= timedelta(days=1)
        if date < min_date:
            # No previous trading day available
            return None
    return date

# Function to get the next trading day
def get_next_trading_day(date):
    max_date = trading_days[-1]
    while date not in trading_days_set:
        date += timedelta(days=1)
        if date > max_date:
            # No next trading day available
            return None
    return date

# Initialize an empty list to collect trades
trades = []

# Fetch data for each stock
for symbol in symbols:
    print(f"\nProcessing {symbol}...")
    # Fetch the stock history with dividends and stock splits
    stock = yf.Ticker(symbol)
    try:
        history = stock.history(start=start_date, end=end_date, actions=True, auto_adjust=False)
        history.index = history.index.tz_localize(None)  # Remove timezone info from index
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        continue

    # Check if history is empty
    if history.empty:
        print(f"No trading data available for {symbol}. Skipping.")
        continue

    # Extract dividend events
    dividends = history[history['Dividends'] != 0]['Dividends']

    # Skip if there are no dividends
    if dividends.empty:
        print(f"No dividends found for {symbol}. Skipping.")
        continue
    else:
        print(f"Found {len(dividends)} dividends for {symbol}")

    # For each dividend event, determine buy and sell dates
    for ex_div_date, dividend_amount in dividends.items():
        # Ensure ex_div_date is a Timestamp
        if not isinstance(ex_div_date, pd.Timestamp):
            ex_div_date = pd.Timestamp(ex_div_date)
        ex_div_date = ex_div_date.normalize()  # Set time to midnight
        ex_div_date = ex_div_date.tz_localize(None)  # Remove timezone info

        # print(f"\nProcessing dividend for {symbol} on ex-div date {ex_div_date.date()} with amount {dividend_amount}")

        # Define buy date (2 business days before ex-dividend date)
        buy_date = ex_div_date - BDay(before_days)
        buy_date = buy_date.normalize()
        buy_date = buy_date.tz_localize(None)
        # print(f"Calculated initial buy_date: {buy_date.date()}")

        # Define sell date (2 business days after ex-dividend date)
        sell_date = ex_div_date + BDay(after_days)
        sell_date = sell_date.normalize()
        sell_date = sell_date.tz_localize(None)
        # print(f"Calculated initial sell_date: {sell_date.date()}")

        # Now check if buy_date and sell_date are in trading_days_set
        if buy_date not in trading_days_set:
            buy_date = get_previous_trading_day(buy_date)
            if buy_date is None:
                # print(f"No trading day available before ex-dividend date for {symbol}. Skipping trade.")
                continue
        # print(f"Adjusted buy_date: {buy_date.date()}")

        if sell_date not in trading_days_set:
            sell_date = get_next_trading_day(sell_date)
            if sell_date is None:
                # print(f"No trading day available after ex-dividend date for {symbol}. Skipping trade.")
                continue
        # print(f"Adjusted sell_date: {sell_date.date()}")

        # Get price at buy date
        try:
            price_at_buy = history.loc[buy_date]['Close']
        except KeyError:
            # print(f"Buy date {buy_date.date()} not in trading data for {symbol}. Skipping trade.")
            continue

        # Calculate dividend yield
        dividend_yield = dividend_amount / price_at_buy

        # Append the trade details to the trades list
        trades.append({
            'symbol': symbol,
            'buy_date': buy_date,
            'sell_date': sell_date,
            'ex_div_date': ex_div_date,
            'dividend_amount': dividend_amount,
            'price_at_buy': price_at_buy,
            'dividend_yield': dividend_yield
        })
        # print(f"Trade added for {symbol}: Buy on {buy_date.date()}, Sell on {sell_date.date()}, Dividend Yield: {dividend_yield:.4%}")

# Convert trades list to DataFrame
trades_df = pd.DataFrame(trades)

print(f"\nTotal trades collected: {len(trades)}")

# Check if trades_df is empty
if trades_df.empty:
    print("No trades were collected. Exiting the simulation.")
    exit()
else:
    # Sort trades by buy_date
    trades_df.sort_values(by='buy_date', inplace=True)


# Initialize variables for simulation
positions = []
total_dividends = 0
portfolio = []
current_date = start_date
end_of_simulation = trades_df['sell_date'].max() if not trades_df.empty else end_date

# Simulate the strategy
while current_date <= end_of_simulation:
    # Check if any positions need to be closed
    positions_to_close = [pos for pos in positions if pos['sell_date'] == current_date]
    for pos in positions_to_close:
        # Fetch sell price
        stock = yf.Ticker(pos['symbol'])
        try:
            history = stock.history(start=pos['sell_date'], end=pos['sell_date'] + timedelta(days=5))
            # Ensure history is not empty
            if history.empty:
                print(f"No trading data available for {pos['symbol']} on sell date {pos['sell_date'].date()}.")
                # Try the next trading day
                adjusted_sell_date = get_next_trading_day(pos['sell_date'] + timedelta(days=1))
                if adjusted_sell_date is None:
                    print(f"No trading day available after sell date for {pos['symbol']}. Skipping closing position.")
                    continue
                history = stock.history(start=adjusted_sell_date, end=adjusted_sell_date + timedelta(days=1))
                if history.empty:
                    print(f"Cannot fetch sell price for {pos['symbol']}. Skipping closing position.")
                    continue
                else:
                    pos['sell_date'] = adjusted_sell_date
            price_at_sell = history['Close'].iloc[0]
        except Exception as e:
            print(f"Error fetching sell price for {pos['symbol']}: {e}")
            continue

        # Update capital
        capital += price_at_sell * pos['shares']
        # Collect dividend
        total_dividends += pos['dividend_amount'] * pos['shares']
        # Remove position
        positions.remove(pos)
        # Record transaction
        portfolio.append({
            'symbol': pos['symbol'],
            'buy_date': pos['buy_date'].date(),
            'sell_date': pos['sell_date'].date(),
            'shares': pos['shares'],
            'buy_price': pos['price_at_buy'],
            'sell_price': price_at_sell,
            'dividend_amount': pos['dividend_amount'],
            'total_dividend': pos['dividend_amount'] * pos['shares']
        })

    # If capital is available and no positions are open
    if not positions:
        # Find the next available trade
        available_trades = trades_df[trades_df['buy_date'] >= current_date]
        if not available_trades.empty:
            next_trade = available_trades.iloc[0]
            # If multiple trades start on the same date, pick the one with the highest dividend yield
            same_day_trades = available_trades[available_trades['buy_date'] == next_trade['buy_date']]
            if len(same_day_trades) > 1:
                next_trade = same_day_trades.loc[same_day_trades['dividend_yield'].idxmax()]
            # Execute the trade
            shares = capital // next_trade['price_at_buy']
            if shares == 0:
                # print(f"Not enough capital to buy any shares of {next_trade['symbol']}")
                current_date += timedelta(days=1)
                continue
            capital -= shares * next_trade['price_at_buy']
            positions.append({
                'symbol': next_trade['symbol'],
                'buy_date': next_trade['buy_date'],
                'sell_date': next_trade['sell_date'],
                'shares': shares,
                'price_at_buy': next_trade['price_at_buy'],
                'dividend_amount': next_trade['dividend_amount']
            })
            # Advance current_date to the buy_date of the executed trade
            current_date = next_trade['buy_date']
            continue

    # Increment current_date by one day
    current_date += timedelta(days=1)

# Calculate total returns
total_invested = 100000
total_value = capital + total_dividends
profit = total_value - total_invested
roi = (profit / total_invested) * 100

print(f"\nTotal invested: ${total_invested:,.2f}")
print(f"Ending capital: ${capital:,.2f}")
print(f"Total dividends collected: ${total_dividends:,.2f}")
print(f"Total portfolio value: ${total_value:,.2f}")
print(f"Profit: ${profit:,.2f}")
print(f"Return on Investment (ROI): {roi:.2f}%")

# Show portfolio transactions
portfolio_df = pd.DataFrame(portfolio)
print("\nPortfolio Transactions:")
# print(portfolio_df)


$DRGO: possibly delisted; no price data found  (1d 2023-01-01 00:00:00 -> 2024-10-01 00:00:00)



Processing JEPQ...
Found 19 dividends for JEPQ

Processing JEPI...
Found 20 dividends for JEPI

Processing SPYI...
Found 20 dividends for SPYI

Processing QQQI...
Found 8 dividends for QQQI

Processing DRGO...
Error fetching data for DRGO: 'Index' object has no attribute 'tz_localize'

Total trades collected: 35

Total invested: $100,000.00
Ending capital: $101,913.82
Total dividends collected: $1,788.30
Total portfolio value: $103,702.12
Profit: $3,702.12
Return on Investment (ROI): 3.70%

Portfolio Transactions:
